<h1>Recursively handling a redundant list of tasks/objects/servers</h1>

Here's a real world example of a task you may want to do recursively using Python exceptions. We'll use the concept of a "redundant" list of objects, where any one of the objects could potentially do the work we want, but some of the objects can't service the request at all at runtime so we need to try the next one, and the next, until we find one that works.

Often, for example, we have a list of servers which may answer our request, but some of the servers may not be working. We want to try each server in turn and only throw an error if none of the servers respond.

We'll start by making a fake server class to stand in for an internet server in this demo. The server is declared as always failing (and throwing an exception) or always working. We also define a couple of custom exceptions, one for when a server fails, and one to use later, if all servers fail.

In [1]:
class ServerException(Exception):
    pass

class AllRedundantFailedException(Exception):
    pass

class Server():
    def __init__(self, name: str, excepts: bool):
        self.name = name
        self.excepts = excepts
    
    def work(self):
        print(self.name + " is trying to do some work...")
        if self.excepts:
            raise ServerException()
        else:
            print("Good server!")

We'll make a couple of good servers and a couple of bad ones for demo purposes.

In [2]:
bad_server = Server("Tom", True)
naughty_server = Server("Dick", True)
good_server = Server("Harry", False)
unused_server = Server("Norman", False)

The servers work (or don't work!) as designed.

In [3]:
try:
    bad_server.work()
except ServerException:
    print("Tom failed.")
    
try:
    naughty_server.work()
except ServerException:
    print("Dick failed.")
    
good_server.work()
unused_server.work()

Tom is trying to do some work...
Tom failed.
Dick is trying to do some work...
Dick failed.
Harry is trying to do some work...
Good server!
Norman is trying to do some work...
Good server!


We could write the redundancy like this, using nested try/except, which is horribly ugly and requires a new clause (and a new level of nesting) for every server we add. Don't do this, this is what we will avoid by using recursion.

In [4]:
try:
    bad_server.work()
except ServerException:
    try:
        naughty_server.work()
    except ServerException:
        try:
            good_server.work()
        except ServerException:
            try:
                unused_server.work()
            except ServerException:
                raise AllRedundantFailedException()
    


Tom is trying to do some work...
Dick is trying to do some work...
Harry is trying to do some work...
Good server!


Here's the much nicer recursive solution. The 'work' method calls itself, with a smaller list of servers each time, until the list is empty, at which point an exception is raised (because no server worked). The 'work' method exits the first time we find a working server.

In [5]:
all_servers = [bad_server, naughty_server, good_server, unused_server]
only_bad_servers = [bad_server, naughty_server]

def work(server_list: list):
    if not server_list:
        raise AllRedundantFailedException()
    server = server_list[0]
    new_server_list = server_list[1:]
    try:
        server.work()
    except:
        work(new_server_list)

try:
    work(all_servers)
except AllRedundantFailedException:
    print("Error: no working server could be found!")

Tom is trying to do some work...
Dick is trying to do some work...
Harry is trying to do some work...
Good server!


If all the servers are bad (i.e. the list becomes empty before a good server is found) an exception is raised:

In [6]:
try:
    work(only_bad_servers)
except AllRedundantFailedException:
    print("Error: no working server could be found!")

Tom is trying to do some work...
Dick is trying to do some work...
Error: no working server could be found!
